In [2]:
# Objective: Practice running R in Python as well as SQL and Spark
# Source: http://lgautier.github.io/odsc-pdaibp-slides/
# Created by: KHS
# Date: 06/19/2018

In [ ]:
# RPy is a simple, easy-to-use interface to R from Python.
#   It enables one to enjoy the elegance of Python programming while having access 
#   to the rich graphical and statistical capabilities of R. 
from rpy2.robjects import r

In [ ]:
# Tables in R
# Make most common-operations in R
# import dplyr

from rpy2.robjects.lib import dplyr


datasrc  = dplyr.src_sqlite(dbfilename)
review_tbl = datasrc.get_table("review")

In [ ]:
# Query as chained call
res =  (review_tbl
        .filter('reviewscore < 2')
        .count('reviewhelpful')
        .arrange('desc(n)'))

print(res)

In [ ]:
# Figures in R
# Objective: show the data in R
# R as a DSL in Python

from rpy2.robjects import r, globalenv
import rpy2.robjects.lib.ggplot2 as gg

split_review = \
    'as.integer(sapply(strsplit(reviewhelpful, "/"), "[", %i))'

p = (gg.ggplot((res.collect().
                mutate(r1 = split_review % 1,
                       r2 = split_review % 2))) +
     gg.aes_string(x='r1+1', y='r2+1', size='n') +
     gg.geom_point(alpha = 0.5) +
     gg.scale_x_log10("review voted helpful") +
     gg.scale_y_log10("review") +
     gg.scale_size(trans="sqrt"))

In [ ]:
from rpy2.robjects.lib import grdevices

with grdevices.render_to_bytesio(grdevices.png, 
                                 width=800,
                                 height=600, 
                                 res=120) as b:
    p.plot()

from IPython.display import Image, display
data = b.getvalue()
display(Image(data=data, format='png', embed=True))

In [ ]:
# What about Javascript then?
# - D3
# - RStudio's Shiny ~ we can call any R
# - ploty (R packaged to plot ggplot2 objects)
# - Continuum's bokeh ~ Python package



In [ ]:
from bokeh.plotting import figure
from bokeh.plotting import figure, show, output_notebook
output_notebook()

plot = figure()
res =  (review_tbl
        .filter('reviewscore < 2')
        .count('reviewhelpful')
        .collect())

import math
plot.scatter(list(int(x.split('/')[0]) \
                  for x in res.rx2('reviewhelpful')),
             list(int(x.split('/')[1]) \
                  for x in res.rx2('reviewhelpful')),
             radius=list(math.log10(x/100) for x in res.rx2('n')),
             fill_alpha=.3)

In [ ]:
show(plot)

In [ ]:
# SPARK
# - Resilient Distributed Dataset (RDD)
# - Abstracts the distribution of compute and data
# - Function composition optimized before evaluation

In [4]:
# PATH = "/Users/ks5gh/Pictures"
# Image(filename = PATH + "pyspark.png", width=100, height=100)

In [ ]:
# Setup
import findspark
findspark.init()

import pyspark

conf = pyspark.conf.SparkConf()
(conf.setMaster('local[2]')
 .setAppName('ipython-notebook')
 .set("spark.executor.memory", "2g"))

sc = pyspark.SparkContext(conf=conf)

In [ ]:
# SWL Table to Spark Dataframe
from pyspark.sql import SQLContext, Row
sqlcontext = SQLContext(sc)
cursor.execute('select * from review limit 10000')
review = \
    sqlcontext.createDataFrame(cursor,
                               tuple(x[0] for x in cursor.description))
review.registerTempTable("review")

In [ ]:
sql = """
SELECT reviewhelpful, count(*) AS ct
FROM review
WHERE reviewscore < 2
GROUP BY reviewhelpful
ORDER BY ct DESC
"""
counts = sqlcontext.sql(sql)

# good ol' SQL
# Translated to Spark, optimized, and to JVM bytecode

In [ ]:
# Lazy Evaluation
res = counts.collect()
res[:3]

In [ ]:
# Map-reduce with spark
names = (review
         .flatMap(lambda p: p.reviewsummary.split())
         .map(lambda word: (word.lower(), 1))
         .reduceByKey(lambda a, b: a+b))
names.takeOrdered(15, key = lambda x: -x[1])

In [ ]:
sql = """
SELECT *
FROM review
WHERE reviewscore < 2
"""
lowscore = sqlcontext.sql(sql)
names = (lowscore
         .flatMap(lambda p: p.reviewsummary.split())
         .map(lambda word: (word.lower(), 1))
         .reduceByKey(lambda a, b: a+b))
names.takeOrdered(8, key = lambda x: -x[1])

In [ ]:
# The Finale
# Python + R + Spark + SQL

lst = names.takeOrdered(8, key = lambda x: -x[1])
from rpy2.robjects.vectors import StrVector, IntVector
dataf = dplyr.DataFrame({'word': StrVector([x[0] for x in lst]),
                         'count': IntVector([x[1] for x in lst])})
p = (gg.ggplot(dataf) +
     gg.geom_bar(gg.aes_string(x='word', y='count'),
                 stat="identity"))

with grdevices.render_to_bytesio(grdevices.png, 
                                 width=800,
                                 height=600, 
                                 res=120) as b:
    p.plot()
display(Image(data=b.getvalue(), format='png', embed=True))

In [ ]:
# Future reads
# - C for runtime speed
# - LLVM for runtime speed
# - Web services

# Advice:
# Use what is alread out there. Be pragmatic.
